#### For Colab

In [17]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [18]:
!nvidia-smi

Fri Mar 12 10:25:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [20]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [6]:
!cd '/content/drive/My Drive/MAGMA: Summarization/transformers/'; pip install -q -e .
!pip install -q -U sentence-transformers
!pip install -q -U wandb
!pip install -q -U rouge-score sacrebleu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 4.1MB/s 
     |████████████████████████████████| 3.2MB 13.1MB/s 
     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 1.2MB 7.3MB/s 
     |████████████████████████████████| 2.0MB 5.4MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 163kB 23.0MB/s 
     |████████████████████████████████| 133kB 20.8MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 61kB 4.9MB/s 


In [7]:
print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

Setting up colab environment
     |████████████████████████████████| 47.5MB 64kB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 3.2MB 34.8MB/s 
     |████████████████████████████████| 1.3MB 30.8MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 204kB 39.6MB/s 
     |████████████████████████████████| 317kB 43.8MB/s 
     |████████████████████████████████| 296kB 41.2MB/s 
     |████████████████████████████████| 143kB 43.3MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 


In [8]:
import sys

print("\n##### sys.path original #####")
for p in sys.path:
    print(f"'{p}'")
print()

import site
site.main()

print("\n##### sys.path after site.main() #####")
for p in sys.path:
    print(f"'{p}'")
print()


##### sys.path original #####
''
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'


##### sys.path after site.main() #####
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'
'/content/drive/My Drive/MAGMA: Summarization/transformers/src'



### **Config**

In [9]:
import sys
sys.path.insert(0, drive_dir)
import config

import wandb
wandb.login()

project_name = 'ft_bart_para_wordembed'
%env WANDB_PROJECT=$project_name

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=ft_bart_para_wordembed


In [10]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'

In [11]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [12]:
data_dir = '"'+drive_dir+'datasets/karger_books_para_wordembed/bart/st/th/"'

source_val_dir = data_dir[:-1] + '/val.source"'
reference_val_dir = data_dir[:-1] + '/val.target"'

source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

In [13]:
GAS = 16
LR = 5e-5

output_dir = '"'+drive_dir+'fine-tuning/ft_bart_para_wordembed_gas'+\
    str(GAS)+'_lr' + str(LR) + '"'

### Evaluation Out of the Box

#### Validation

In [14]:
eval_model_name_or_path = '"sshleifer/distilbart-cnn-12-6"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_para_wordembed_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_para_wordembed_val.json"'
print(save_dir)

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_bart_para_wordembed_gas16_lr5e-05/sshleifer?distilbart-cnn-12-6_para_wordembed_val.txt"
100% 306M/306M [00:47<00:00, 6.50MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
Downloading: 100% 1.62k/1.62k [00:00<00:00, 1.22MB/s]
Downloading: 100% 1.22G/1.22G [00:28<00:00, 43.4MB/s]
Downloading: 100% 899k/899k [00:01<00:00, 843kB/s]
Downloading: 100% 456k/456k [00:00<00:00, 507kB/s]
Downloading: 100% 26.0/26.0 [00:00<00:00, 22.2kB/s]
  0% 0/34 [00:00<?, ?it/s]2021-03-12 09:58:10.417974: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [01:23<00:00,  2.47s/it]
{'rouge1_precision': 18.2, 'rouge1_recall': 38.98, 'rouge1_fmeasure': 23.49, 'rouge2_precision': 5.46, 'rouge2_recall': 12.920000000000002, 'rouge2_fmeasure': 7.290000000000001, 'rougeL_precision': 13.0, 'rougeL_recall': 28.65, 'rougeL_fmeasure': 16.97, 'rougeLsum_p

#### Test

In [15]:
eval_model_name_or_path = '"sshleifer/distilbart-cnn-12-6"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_para_wordembed_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_para_wordembed_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/31 [00:00<?, ?it/s]2021-03-12 10:00:16.718728: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 31/31 [01:17<00:00,  2.51s/it]
{'rouge1_precision': 16.68, 'rouge1_recall': 38.190000000000005, 'rouge1_fmeasure': 22.07, 'rouge2_precision': 5.3, 'rouge2_recall': 12.379999999999999, 'rouge2_fmeasure': 7.01, 'rougeL_precision': 12.22, 'rougeL_recall': 28.549999999999997, 'rougeL_fmeasure': 16.3, 'rougeLsum_precision': 14.030000000000001, 'rougeLsum_recall': 32.550000000000004, 'rougeLsum_fmeasure': 18.64, 'sentence_distilroberta_cosine': 48.52704107761383, 'w2v_cosine': 99.99573230743408, 'n_obs': 242, 'runtime': 77, 'seconds_per_sample': 0.3182, 'min_length': 10, 'max_length': 150}


### Fine-Tuning

In [ ]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['num_beams'] = 2
model_config_dir = '"'+drive_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--freeze_embeds \
--do_train \
--learning_rate 5e-5 \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--fp16 \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--num_train_epochs 20 \
--logging_steps 10 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps 16 \
--evaluation_strategy steps --eval_steps 7 --eval_beams 2 \
--predict_with_generate \
--save_steps 98 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

Streaming output truncated to the last 5000 lines.
Batches: 100%|██████████| 1/1 [00:00<00:00, 154.06it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 151.17it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 149.99it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 146.27it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 152.49it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 149.41it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 146.79it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 156.60it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 158.29it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 155.48it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 158.56it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 152.91it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 159.06it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 155.31it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 158.07it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 154.12it/s]


Batches: 100%|███████

### Evaluation

In [21]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-196"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_para_wordembed_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_para_wordembed_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_bart_para_wordembed_gas16_lr5e-05/checkpoint-196"

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/34 [00:00<?, ?it/s]2021-03-12 10:26:48.552608: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [01:13<00:00,  2.16s/it]
{'rouge1_precision': 20.06, 'rouge1_recall': 42.14, 'rouge1_fmeasure': 25.16, 'rouge2_precision': 6.569999999999999, 'rouge2_recall': 14.59, 'rouge2_fmeasure': 8.33, 'rougeL_precision': 14.66, 'rougeL_recall': 31.240000000000002, 'rougeL_fmeasure': 18.43, 'rougeLsum_precision': 16.25, 'rougeLsum_recall': 34.58, 'rougeLsum_fmeasure': 20.46, 'sentence_distilroberta_cosine': 59.32483673095703, 'w2v_cosine': 99.99169111251831, 'n_obs': 267, 'runtime': 73, 'seconds_per_sample': 0.2734, 'min_length': 10, 'max_length': 150}


In [22]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-196"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_para_wordembed_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_para_wordembed_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_bart_para_wordembed_gas16_lr5e-05/checkpoint-196"

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/31 [00:00<?, ?it/s]2021-03-12 10:28:52.865397: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 31/31 [01:06<00:00,  2.15s/it]
{'rouge1_precision': 18.59, 'rouge1_recall': 41.27, 'rouge1_fmeasure': 23.82, 'rouge2_precision': 6.619999999999999, 'rouge2_recall': 14.12, 'rouge2_fmeasure': 8.309999999999999, 'rougeL_precision': 13.819999999999999, 'rougeL_recall': 30.79, 'rougeL_fmeasure': 17.72, 'rougeLsum_precision': 15.079999999999998, 'rougeLsum_recall': 33.739999999999995, 'rougeLsum_fmeasure': 19.33, 'sentence_distilroberta_cosine': 56.97326064109802, 'w2v_cosine': 99.99489188194275, 'n_obs': 242, 'runtime': 66, 'seconds_per_sample': 0.2727, 'min_length': 10, 'max_length': 150}
